# Automatická detekcia TLE na snímkach AMOS pomocou hlbokého učenia

## Príprava prostredia

Príkazy na inštaláciu potrebných knižníc.

In [ ]:
!pip install PyPDF2
!pip install pdf2image
!conda install -c conda-forge poppler -y
!pip install os-sys
!pip install temp
!pip install zipfile38
!pip install pycopy-fnmatch
!pip install python-csv
!pip install pandas
!pip install numpy
!pip install jsonlib
!pip install python-math
!pip install opencv-python
!pip install Pillow

Import potrebných knižníc.

In [ ]:
import PyPDF2
import os
import tempfile
import zipfile,fnmatch
import csv
import pandas as pd
import numpy as np
import json
import math 
import cv2
from PIL import Image, ImageDraw
import operator
from skimage.io import imread
from PyPDF2 import PdfFileWriter, PdfFileReader
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
from PIL import Image

## Vytvorenie prvotnej tabuľky a vykreslenie anotácií

data: cesta k súboru, ktorý sme dostali ako export zo Zooniverze <br>
output: cesta, kde bude uložená tabuľka len s anotovanými obrázkami <br>
rootPath: cesta, kde sú uložené obrázky

In [ ]:
data = pd.read_csv("detekcia-tle-2-classifications.csv") 
output="output5.csv"
rootPath = r"files"
with open(output, mode='w') as csv_file:
    columns=["is_tle","event","subject_id","x","y","width","height","type"]
    write=csv.DictWriter(csv_file,fieldnames=columns)
    write.writeheader()
    i=0
    while (i<len(data)):
        is_tle=json.loads(data.iloc[i,11])[0]["value"]
        if is_tle=="Áno":
            anot=json.loads(data.iloc[i,11])[1]["value"]
            pocet=0
            while(pocet<len(anot)):
                event=json.loads(data.iloc[i,12])[str(data.iloc[i,13])]["Filename"]
                subject_id=json.loads(str(data.iloc[i,13]))
                x=json.loads(data.iloc[i,11])[1]["value"][pocet]["x"]
                y=json.loads(data.iloc[i,11])[1]["value"][pocet]["y"]
                width=json.loads(data.iloc[i,11])[1]["value"][pocet]["width"]
                height=json.loads(data.iloc[i,11])[1]["value"][pocet]["height"]
                type_tle=json.loads(data.iloc[i,11])[1]["value"][pocet]["tool_label"]
                write.writerow({'is_tle': is_tle,'event': event,'subject_id': subject_id,'x': x,'y':y,'width':width,'height':height,'type':type_tle})
                pocet+=1
            for root, dirs, files in os.walk(rootPath):
                for file in files:
                    if file == event:
                        im = Image.open(os.path.join(root, "%s" % (file)))
                        im1 = ImageDraw.Draw(im)
                        pocet=0
                        while(pocet<len(anot)):
                            if (type_tle == 'Mrkva (Carrot sprite)'):
                                shape=[(x, y), (x, y+height)]
                                im1.line(shape, fill="orange", width=2)
                                shape=[(x, y+height),(x+width,y+height) ]
                                im1.line(shape, fill="orange", width=2)
                                shape=[(x, y), (x+width, y)]
                                im1.line(shape, fill="orange", width=2)
                                shape=[(x+width, y),(x+width,y+height) ]
                                im1.line(shape, fill="orange", width=2)
                            elif (type_tle == 'Stĺp (Column sprite)'):
                                shape=[(x, y), (x, y+height)]
                                im1.line(shape, fill="fuchsia", width=2)
                                shape=[(x, y+height),(x+width,y+height) ]
                                im1.line(shape, fill="fuchsia", width=2)
                                shape=[(x, y), (x+width, y)]
                                im1.line(shape, fill="fuchsia", width=2)
                                shape=[(x+width, y),(x+width,y+height) ]
                                im1.line(shape, fill="fuchsia", width=2)
                            elif (type_tle == 'Fontána (Jet, Starter)'):
                                shape=[(x, y), (x, y+height)]
                                im1.line(shape, fill="cyan", width=2)
                                shape=[(x, y+height),(x+width,y+height) ]
                                im1.line(shape, fill="cyan", width=2)
                                shape=[(x, y), (x+width, y)]
                                im1.line(shape, fill="cyan", width=2)
                                shape=[(x+width, y),(x+width,y+height) ]
                                im1.line(shape, fill="cyan", width=2)
                            pocet+=1
                            print("File %s done." % (event))
                            im.save(os.path.join('annotations', "1%s" % (file))) 
                            
            
        i+=1




## Vytvorenie textových súborov s normalizovanými súradnicami stredu anotácie

data: cesta k súboru, ktorý sme dostali ako export zo Zooniverze <br>
rootPath: cesta, kde sú uložené vykreslené obrázky <br>
textFile: cesta, kam sa uložia textové súbory

In [ ]:
data = pd.read_csv("detekcia-tle-2-classifications.csv") 
rootPath = r"annotations"
textFile=os.getcwd()+"/txt1"
i=0
cislo=0
while (i<len(data)):
    is_tle=json.loads(data.iloc[i,11])[0]["value"]
    if is_tle=="Áno":
        anot=json.loads(data.iloc[i,11])[1]["value"]
        event=json.loads(data.iloc[i,12])[str(data.iloc[i,13])]["Filename"].split(".")
        pocet=0
        with open(os.path.join(textFile, "%s.txt" % (event[0])), mode='w') as csv_file:
            
            while(pocet<len(anot)):
                
                subject_id=json.loads(str(data.iloc[i,13]))
                x=json.loads(data.iloc[i,11])[1]["value"][pocet]["x"]
                y=json.loads(data.iloc[i,11])[1]["value"][pocet]["y"]
                width=json.loads(data.iloc[i,11])[1]["value"][pocet]["width"]
                height=json.loads(data.iloc[i,11])[1]["value"][pocet]["height"]
                type_tle=json.loads(data.iloc[i,11])[1]["value"][pocet]["tool_label"]
                #ak by sme chceli rozdelit druhy TLE javov odkomentujeme nasledujuci kod
                #if(type_tle == 'Mrkva (Carrot sprite)'):
                #    cislo=0
                #elif (type_tle == 'Stĺp (Column sprite)'):
                #    cislo=1
                #elif (type_tle == 'Fontána (Jet, Starter)'):
                #    cislo=2
                fieldnames1 = [cislo,round((x+(width/2))/1280,5),round((y+(height/2))/960,5),round(width/1280,5),round(height/960,5)]
                writer = csv.DictWriter(csv_file,fieldnames=fieldnames1, delimiter=' ')
                writer.writeheader()
                print("Just made normalized txt file for %s" % (event[0]))
                pocet+=1
    i+=1
            
            
            
